# Code used to read the data used for finetuning the BERT model

In [ ]:
import sqlite3
from sqlite3 import Error


path = r"D:\release_db.sqlite"
#cur.execute("SELECT * FROM policy_texts")

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

conn = create_connection(path)
conn.text_factory = lambda b: b.decode(errors = 'ignore')
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM policy_texts")

example = cur.fetchone()
print(len(example))
print(type(example))
print("text is example[1]")
extext = example[1]

In [ ]:
cur.description

In [ ]:
example

# Create subdataset of text. Only 10.000 texts

In [ ]:
import csv

n_texts = 10000
rows = cur.fetchmany(n_texts)

csv_name = "policy_subdata"
colnames = ["id","policy_text","length","flesch_kincaid"]

with open(csv_name + '.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(colnames)

for row in rows:
    pol_id = row[0]
    pol_text = row[1]
    pol_l = row[7]
    pol_fk = row[2]
    new_row = [pol_id,pol_text,pol_l,pol_fk]
    with open(csv_name + '.csv', 'a', encoding='utf-8') as g:
        writer = csv.writer(g)
        writer.writerow(new_row)

In [ ]:
#rows = cur.fetchall()

newpath =  r"D:\raw_texts"

for row in rows:
    pol_id = row[0]
    pol_text = row[1]
    filename = "/" + str(pol_id) + ".txt"
    filepath = newpath + filename
    afile = open(filepath,"w", encoding='utf-8') 
    afile.write(pol_text)
    afile.close()

In [ ]:
a = b'\x8b'
b = a.decode("iso-8859-1")
print(b)

In [ ]:
split_for_dots = extext.split('.')
split_for_nn = extext.split('\n\n')
split_for_space = extext.split(' ')

In [ ]:
print(len(split_for_dots))
print(len(split_for_nn))
print(len(split_for_space))


In [ ]:
from transformers import BertConfig, BertTokenizerFast
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

In [ ]:
def functok(text_in):
    return tokenizer(
    text=text_in,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
max_length = 128
x = tokenizer(
    text=split_for_nn,
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
rows = cur.fetchall()

In [ ]:
n_rows = len(rows)
lenlist = []
count = 0
for row in rows:
    text = row[1]
    split_for_nn = text.split('\n\n')
    for segment in split_for_nn:
        size = functok(segment)["input_ids"].shape[1]
        lenlist.append(size)
    
    count+= 1
    if count % 10000 == 0:
        print("finished with 10000 docs, total count: ",count)
        print("percentag done: ",round(count / n_rows,3),"%")

    

In [ ]:
from collections import Counter

len_counter = Counter(lenlist)

In [ ]:
cur2 = conn.cursor()
cur2.execute("SELECT * FROM policy_texts WHERE rowid = 2")
a = cur2.fetchone()

In [ ]:
seg = 0
i=0
rowfound = None
while seg < 548:
    row = rows[i]
    text = row[1]
    split_for_nn = text.split('\n\n')
    split_len = len(split_for_nn)
    seg += split_len
    rowfound = row
    
    i+=1

# Turning seperate texts into one big .txt file

In [1]:
from pathlib import Path
import nltk
from tensorflow.keras.utils import Progbar

regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
    # lowercase text
    text = str(text).lower()
    # remove non-UTF
    text = text.encode("utf-8", "ignore").decode()
    # remove punktuation symbols
    
    tokenized = regex_tokenizer.tokenize(text)
    if len(tokenized) > 4:
        text = " ".join(tokenized)
        return text
    else:
        return ''



In [2]:
def count_files(apath):
    genob = Path(apath).glob("**/*.txt")
    count = 0
    for file in genob:
        count += 1
    return count

In [5]:
i = 0

OUTPUTNAME = "bigtext.txt"
DIRNAME = "./data/subsub"


outputfile = open(OUTPUTNAME,"w")
outputfile.close()

src_files = Path("./data/raw_texts").glob("**/*.txt")
outputfile = open(OUTPUTNAME,"a", encoding="utf-8") #append mode
    
n_files = count_files("./data/raw_texts")
bar = Progbar(n_files)
    
for src_file in src_files:
    #print("🔥", src_file)
    text = src_file.read_text(encoding="utf-8")
    lines = text.split("\n\n")
    for line in lines:
        sents = line.split('.')
        for sent in sents:
            norm_sent = normalize_text(sent)
            if norm_sent:
                outputfile.write(norm_sent + " \n")
                #print(norm_sent + "\n")
                i +=1
    bar.add(1)
                    
outputfile.close()


9999/9999 [==============================] - 37s 4ms/step


In [42]:
def count_lines2(filename):
    count = 0
    with open(filename, encoding = 'utf-8') as fi:
        for line in fi:
            count += 1
    return count

print(count_lines2(OUTPUTNAME))

9603994


In [12]:
from nltk import RegexpTokenizer, word_tokenize

In [1]:
from pathlib import Path
import nltk
from tensorflow.keras.utils import Progbar

In [10]:
def count_files(apath):
    genob = Path(apath).glob("**/*.txt")
    count = 0
    for file in genob:
        count += 1
    return count

In [13]:
OUTPUTNAME = r"D:\bigger.txt"
#outputfile = open(OUTPUTNAME,"w")
#outputfile.close()


src_files = Path(r"D:\raw_texts").glob("**/*.txt")
outputfile = open(OUTPUTNAME,"a", encoding="utf-8") #append mode
    
n_files = count_files(r"D:\raw_texts")
bar = Progbar(n_files)

tok_lengths = []

ind = 0
for src_file in src_files:
    if ind <= poss:
        pass
    else:
    #print("🔥", src_file)
        text = src_file.read_text(encoding="utf-8")
        lines = text.split("\n\n")
        for line in lines:
            toktline = word_tokenize(line)
            toklen = len(toktline)
            if toklen > 4:
                tok_lengths.append(toklen)
                strline = ' '.join(toktline)
                outputfile.write(strline + " \n")
            else:
                pass

    ind +=1
    
        
    bar.add(1)
                    
outputfile.close()

333985/333985 [==============================] - 12386s 37ms/step


In [26]:
output_name = r"D:\tok_lenghts.csv"

df.to_csv(output_name)